In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import openpyxl
from config.want_type import *
from PyQt5.QtWidgets import *# 메인 루프
import os
import sys
import time

ModuleNotFoundError: No module named 'config'

In [6]:
class Want_code():
    want_area = [
                "전자장비와기기",
                "디스플레이패널",
                "반도체와반도체장비",
                "소프트웨어",
                "인터넷과카탈로그소매",
                "게임엔터테인먼트",
                "우주항공과국방",
                "양방향미디어와서비스",
                "통신장비",
                "IT서비스",
                "컴퓨터와주변기기",
                "무선통신서비스",
                "디스플레이장비및부품",
                "전기제품",
             ]

In [7]:
class CherryPick():
    def __init__(self):
        self.krx_code_list = [] #외부로부터 들고올 모든 종목의 종목코드 리스트
        self.financial_dict = {} #크롤링을 통해 재무제표를 들고올 리스트
        self.get_code()
        self.stock_info()
        #### 외부클래스/변수 ###원하는 업종 따로 정리 
        want_code = Want_code()
        self.want_area_list = want_code.want_area #원하는 업종리스트
    def get_code(self):
        self.df = pd.read_excel("KrxCode.xlsx")
        self.krx_code_list = self.df["단축코드"].values
    def stock_info(self): # 개별 종목코드의 재무제표를 얻는다
        idx = 0
        for code in self.krx_code_list:
            idx+=1
            #print("{}/{} 진행중".format(idx,len(self.krx_code_list))))
            self.crawl_info(code)
    def crawl_info(self,code): # 개별 종목코드의 재무제표를 얻기위해 크롤링을 수행하는 메소드
        try:
            url = "https://finance.naver.com/item/main.nhn?code=" + code
            html = requests.get(url)
            soup = BeautifulSoup(html.content, "html.parser")
            code_nm = self.df[self.df["단축코드"] == code]["한글 종목명"].values[0]
            area = soup.select("div > h4 > em > a")[0].text
            area_per = float(soup.select("tr.strong > td > em")[2].text)
            cap = soup.select('table[class = "tb_type1 tb_num"]')[0].select("tbody > tr")[3].find("td").text
            if "," in cap:
                cap = int(re.sub(",", "", cap))
            else:
                cap = int(cap)
            per = soup.select('table[class = "tb_type1 tb_num"]')[0].select("tbody > tr")[12].find("td").text
            if per is None:
                per = 0
            else:
                per = float(per)
            pbr = soup.select('table[class = "tb_type1 tb_num"]')[0].select("tbody > tr")[13].find("td").text
            if pbr is None:
                pbr = 0
            else:
                pbr = float(pbr)
            tr = soup.find("table", {"class": "tb_type1 tb_num tb_type1_ifrs"}).select("tbody > tr")
            if tr is not None:
                total_list = []
                for i in np.array([0,1,2,3,5,6,7]):
                    td = tr[i].findAll("td")
                    data= td[8].text.strip()
                    if "," in data:
                        data = float(re.sub(",", "", data))
                        total_list.append(data)
                    elif data == None:
                        total_list.append(0)
                    else:
                        total_list.append(float(data))
                if code not in self.financial_dict:
                    self.financial_dict.update({code:{}})
                self.financial_dict[code].update({
                                                 "종목명" : code_nm,
                                                  "시가총액": cap,
                                                 "업종" : area,
                                                 "업종PER" : area_per,
                                                 "매출액" : total_list[0],
                                                  "영업이익" : total_list[1],
                                                  "당기손익" : total_list[2],
                                                  "영업이익률" : total_list[3],
                                                  "ROE" : total_list[4],
                                                  "부채비율" : total_list[5],
                                                  "당좌비율" : total_list[6],
                                                  "PER" : per,
                                                  "PBR" : pbr
                })
        except :
            pass
    def to_xlsx(self):
        if not os.path.exists("CherryPick_KR.xlsx"):
            wb = openpyxl.Workbook()
            sheet1 = wb.active
            sheet2 = wb.create_sheet()
            sheet3 = wb.create_sheet()
            sheet1.title = "전종목"
            sheet2.title = "선택업종"
            sheet3.title = "저PER고ROE"
            sheet1.append(["종목코드", "종목명", "시가총액", "PER", "PBR",
                           "업종", "업종PER", "매출액", "영업이익", "당기손익",
                           "영업이익률", "ROE", "당좌비율", "부채비율", "기준_최근분기"])
            sheet2.append(["종목코드", "종목명", "시가총액", "PER", "PBR",
                           "업종", "업종PER", "매출액", "영업이익", "당기손익",
                           "영업이익률", "ROE", "당좌비율", "부채비율", "기준_최근분기"])
            sheet3.append(["종목코드", "종목명", "시가총액", "PER", "PBR",
                           "업종", "업종PER", "매출액", "영업이익", "당기손익",
                           "영업이익률", "ROE", "당좌비율", "부채비율", "기준_최근분기"])
        else :
            wb = openpyxl.load_workbook("CherryPick_KR.xlsx")
            sheet1 = wb.active
            sheet2 = wb["선택업종"]
            sheet3 = wb["저PER고ROE"]
        idx = 0
        for code in self.financial_dict.keys():
            idx += 1
            print("{}/{} 진행중".format(idx, len(self.financial_dict)))
            code_nm = self.financial_dict[code]["종목명"]

